In [15]:
import json 
import os
import re
import html
import pickle

import numpy as np
import pandas as pd
import tokenize_uk

from fastai.text import *

from sklearn.model_selection import train_test_split

In [4]:
BOS = 'xbos'  # beginning-of-sentence tag
FLD = 'xfld'  # data field tag

In [20]:
path = '../../data/ukwiki'

lm_files = [] 
for d in os.listdir(path):
    wiki_files = os.listdir(os.path.join(path, d))
    for f in wiki_files: 
        lm_files.append(os.path.join(path, d, f))

In [21]:
print('Num files:', len(lm_files))
lm_files[0:10]

Num files: 2116


['../../data/ukwiki/AV/wiki_11',
 '../../data/ukwiki/AV/wiki_10',
 '../../data/ukwiki/AV/wiki_03',
 '../../data/ukwiki/AV/wiki_04',
 '../../data/ukwiki/AV/wiki_05',
 '../../data/ukwiki/AV/wiki_02',
 '../../data/ukwiki/AV/wiki_15',
 '../../data/ukwiki/AV/wiki_12',
 '../../data/ukwiki/AV/wiki_13',
 '../../data/ukwiki/AV/wiki_14']

In [22]:
texts = []
for i in lm_files:
    with open(i) as f:
        for line in f:
            texts.append(json.loads(line))
        
texts = pd.DataFrame(texts)

In [23]:
texts.head()

,id,text,title,url
0,2705951,"ГЕС Алматті I, II\n\nГЕС Алматті I, II — гідро...","ГЕС Алматті I, II",https://uk.wikipedia.org/wiki?curid=2705951
1,2705954,Цифровий відбиток пристрою\n\nЦифровий відбито...,Цифровий відбиток пристрою,https://uk.wikipedia.org/wiki?curid=2705954
2,2705955,Наїзд фургона на натовп у Торонто (2018)\n\nО ...,Наїзд фургона на натовп у Торонто (2018),https://uk.wikipedia.org/wiki?curid=2705955
3,2705963,Бавер Лідія Миколаївна\n\nЛі́дія Микола́ївна Б...,Бавер Лідія Миколаївна,https://uk.wikipedia.org/wiki?curid=2705963
4,2705966,360 Total Security\n\n360 Total Security — ком...,360 Total Security,https://uk.wikipedia.org/wiki?curid=2705966


In [24]:
def split_title_from_text(text):
    words = text.split("\n\n")
    if len(words) >= 2:
        return ''.join(words[1:])
    else:
        return ''.join(words)
    
texts['text'] = texts['text'].apply(lambda x: split_title_from_text(x))

In [25]:
texts.head()

,id,text,title,url
0,2705951,"ГЕС Алматті I, II — гідроелектростанції на пів...","ГЕС Алматті I, II",https://uk.wikipedia.org/wiki?curid=2705951
1,2705954,Цифровий відбиток пристрою (англ. fingerprint ...,Цифровий відбиток пристрою,https://uk.wikipedia.org/wiki?curid=2705954
2,2705955,"О 13:30 (UTC -4:00), 23 квітня 2018, арендован...",Наїзд фургона на натовп у Торонто (2018),https://uk.wikipedia.org/wiki?curid=2705955
3,2705963,"Лі́дія Микола́ївна Ба́вер ( м. Онєга, Архангел...",Бавер Лідія Миколаївна,https://uk.wikipedia.org/wiki?curid=2705963
4,2705966,360 Total Security — комплексний антивірусний ...,360 Total Security,https://uk.wikipedia.org/wiki?curid=2705966


In [26]:
texts.shape

(784390, 4)

In [27]:
texts.head()

,id,text,title,url
0,2705951,"ГЕС Алматті I, II — гідроелектростанції на пів...","ГЕС Алматті I, II",https://uk.wikipedia.org/wiki?curid=2705951
1,2705954,Цифровий відбиток пристрою (англ. fingerprint ...,Цифровий відбиток пристрою,https://uk.wikipedia.org/wiki?curid=2705954
2,2705955,"О 13:30 (UTC -4:00), 23 квітня 2018, арендован...",Наїзд фургона на натовп у Торонто (2018),https://uk.wikipedia.org/wiki?curid=2705955
3,2705963,"Лі́дія Микола́ївна Ба́вер ( м. Онєга, Архангел...",Бавер Лідія Миколаївна,https://uk.wikipedia.org/wiki?curid=2705963
4,2705966,360 Total Security — комплексний антивірусний ...,360 Total Security,https://uk.wikipedia.org/wiki?curid=2705966


#### limit corpus to 100 million of records

In [28]:
texts['labels'] = 0
texts = texts[['labels', 'text']]
texts['len'] = texts['text'].apply(lambda x: len(tokenize_uk.tokenize_words(x)))


In [37]:
texts = texts[texts['len'] > 600]

In [38]:
texts.shape

(72728, 3)

#### train test split

In [39]:
train_texts, val_texts = train_test_split(texts, test_size=0.1, random_state=1234, shuffle=True)

In [47]:
train_texts[['labels', 'text']].to_csv('../../data/train.csv', header=False, index=False)
val_texts[['labels', 'text']].to_csv('../../data/val.csv', header=False, index=False)

In [5]:
class UKTokenizer():
    def __init__(self):
        self.re_br = re.compile(r'<\s*br\s*/?>', re.IGNORECASE)

    def sub_br(self,x): 
        return self.re_br.sub("\n", x)

    def tokenize(self, x): 
        return tokenize_uk.tokenize_words(self.sub_br(x))

    re_rep = re.compile(r'(\S)(\1{3,})')
    re_word_rep = re.compile(r'(\b\w+\W+)(\1{3,})')

    @staticmethod
    def replace_rep(m):
        TK_REP = 'tk_rep'
        c,cc = m.groups()
        return f' {TK_REP} {len(cc)+1} {c} '

    @staticmethod
    def replace_wrep(m):
        TK_WREP = 'tk_wrep'
        c,cc = m.groups()
        return f' {TK_WREP} {len(cc.split())+1} {c} '

    @staticmethod
    def do_caps(ss):
        TOK_UP,TOK_SENT,TOK_MIX = ' t_up ',' t_st ',' t_mx '
        res = []
        prev='.'
        re_word = re.compile('\w')
        re_nonsp = re.compile('\S')
        for s in re.findall(r'\w+|\W+', ss):
            res += ([TOK_UP,s.lower()] if (s.isupper() and (len(s)>2))
    #                 else [TOK_SENT,s.lower()] if (s.istitle() and re_word.search(prev))
                    else [s.lower()])
    #         if re_nonsp.search(s): prev = s
        return ''.join(res)

    def proc_text(self, s):
        s = self.re_rep.sub(UKTokenizer.replace_rep, s)
        s = self.re_word_rep.sub(UKTokenizer.replace_wrep, s)
        s = UKTokenizer.do_caps(s)
        s = re.sub(r'([/#])', r' \1 ', s)
        s = re.sub(' {2,}', ' ', s)
        return self.tokenize(s)

    @staticmethod
    def proc_all(ss):
        tok = UKTokenizer()
        return [tok.proc_text(s) for s in ss]

    @staticmethod
    def proc_all_mp(ss):
        ncpus = num_cpus()//2
        with ProcessPoolExecutor(ncpus) as e:
            return sum(e.map(UKTokenizer.proc_all, ss), [])

In [6]:
re1 = re.compile(r'  +')

def fixup(x):
    x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>','u_n').replace(' @.@ ','.').replace(
        ' @-@ ','-').replace('\\', ' \\ ')
    return re1.sub(' ', html.unescape(x))

def get_texts(df, n_lbls=1):
    labels = df.iloc[:,range(n_lbls)].values.astype(np.int64)
    texts = f'\n{BOS} {FLD} 1 ' + df[n_lbls].astype(str)
    for i in range(n_lbls+1, len(df.columns)): texts += f' {FLD} {i-n_lbls} ' + df[i].astype(str)
    texts = texts.apply(fixup).values.astype(str)

    tok = UKTokenizer().proc_all_mp(partition_by_cores(texts))
    return tok, list(labels)

def get_all(df, n_lbls):
    tok, labels = [], []
    for i, r in enumerate(df):
        print(i)
        tok_, labels_ = get_texts(r, n_lbls)
        tok += tok_;
        labels += labels_
    return tok, labels

In [7]:
chunksize=10000
train_texts = pd.read_csv('../../data/train.csv', header=None, chunksize=chunksize)
val_texts = pd.read_csv('../../data/val.csv', header=None, chunksize=chunksize)

In [8]:
tok_trn, trn_labels = get_all(train_texts, 1)
tok_val, val_lables = get_all(val_texts, 1)

0
1
2
3
4
5
6
0


In [9]:
freq = Counter(p for o in tok_trn for p in o)
freq.most_common(25)

[(',', 6055767),
 ('.', 5475994),
 ('в', 1648059),
 ('і', 1460758),
 ('у', 1387202),
 ('на', 1250826),
 ('з', 1153768),
 (')', 965364),
 ('(', 957776),
 ('«', 902154),
 ('»', 899536),
 ('-', 816885),
 ('—', 788103),
 ('та', 766387),
 ('до', 684830),
 ('t_up', 607450),
 ('"', 584384),
 ('що', 574805),
 ('за', 434976),
 ('року', 419553),
 ('не', 380220),
 (':', 347002),
 ('а', 331484),
 ('для', 329646),
 ('як', 284788)]

In [10]:
max_vocab = 60000
min_freq = 2

itos = [o for o, c in freq.most_common(max_vocab) if c>min_freq] # getting rid of the rare words
itos.insert(0, '_pad_')
itos.insert(0, '_unk_')

In [11]:
stoi = collections.defaultdict(lambda:0, {v:k for k,v in enumerate(itos)})
len(itos)

60002

In [13]:
trn_lm = np.array([[stoi[o] for o in p] for p in tok_trn])
val_lm = np.array([[stoi[o] for o in p] for p in tok_val])

In [16]:
np.save('../../data/trn_ids.npy', trn_lm)
np.save('../../data/val_ids.npy', val_lm)
with open('../../data/itos.pkl', 'wb') as f:
    pickle.dump(itos, f)

In [14]:
vs=len(itos)
vs,len(trn_lm)

(60002, 65455)

## Language model

In [33]:
em_sz,nh,nl = 400,1150,3

wd=1e-7
bptt=70
bs=52
opt_fn = partial(optim.Adam, betas=(0.8, 0.99))

model_path = '../../bin/uk_lm'

In [34]:
trn_dl = LanguageModelLoader(np.concatenate(trn_lm), bs, bptt)
val_dl = LanguageModelLoader(np.concatenate(val_lm), bs, bptt)
md = LanguageModelData(model_path, 1, vs, trn_dl, val_dl, bs=bs, bptt=bptt)

In [35]:
drops = np.array([0.25, 0.1, 0.2, 0.02, 0.15])*0.7

In [36]:
learner= md.get_model(opt_fn, em_sz, nh, nl, 
    dropouti=drops[0], dropout=drops[1], wdrop=drops[2], dropoute=drops[3], dropouth=drops[4])

learner.metrics = [accuracy]
learner.clip = 0.2
learner.unfreeze()

In [37]:
lr=1e-3
lrs = lr

In [ ]:
learner.fit(lrs/2, 1, wds=wd, use_clr=(32,2), cycle_len=1)

In [ ]:
learner.save('lm_ukrainian')